In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
%matplotlib inline

In [2]:
master = pd.read_csv('data/Master.csv', parse_dates=['debut', 'finalGame'])
# Currently not caring about pitchers

In [3]:
bbmaster = master[['playerID', 'nameFirst', 'nameLast', 'nameGiven']]
print(len(bbmaster))
bbmaster.head(2)

18589


,playerID,nameFirst,nameLast,nameGiven
0,aardsda01,David,Aardsma,David Allan
1,aaronha01,Hank,Aaron,Henry Louis


In [4]:
batting = pd.read_csv('data/Batting.csv')
batting.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [5]:
print(len(batting))
batting.head(2)

99846


,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13,8,1,4,0,NaN,NaN,NaN,NaN,NaN


In [6]:
master_batting = pd.merge(master, batting)

In [7]:
batting.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [8]:
bats = batting[['playerID', 'yearID','AB', 'H', 'BB', 'HBP', 'SF']]
bats.head()
print(len(bats))
bbdata2 = pd.merge(bbmaster, bats)
bbdata2.head(2)

99846


,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF
0,aardsda01,David,Aardsma,David Allan,2004,0,0,0,0,0
1,aardsda01,David,Aardsma,David Allan,2006,2,0,0,0,0


In [9]:
salaries = pd.read_csv('data/Salaries.csv')
salaries.columns

Index(['yearID', 'teamID', 'lgID', 'playerID', 'salary'], dtype='object')

In [10]:
salaries = salaries[['yearID', 'playerID', 'salary']]
print(len(salaries))
salaries.head(2)

24758


,yearID,playerID,salary
0,1985,barkele01,870000
1,1985,bedrost01,550000


In [11]:
bbdata_sal = pd.merge(bbdata2, salaries)
print(len(bbdata_sal))
bbdata_sal.head()

26470


,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary
0,aardsda01,David,Aardsma,David Allan,2004,0,0,0,0,0,300000
1,aardsda01,David,Aardsma,David Allan,2007,0,0,0,0,0,387500
2,aardsda01,David,Aardsma,David Allan,2008,1,0,0,0,0,403250
3,aardsda01,David,Aardsma,David Allan,2009,0,0,0,0,0,419000
4,aardsda01,David,Aardsma,David Allan,2010,0,0,0,0,0,2750000


In [12]:
# Drop all AB values of 0 or null (average at-bats in MLB is 30/year)
bbdata_sal = bbdata_sal.drop(bbdata_sal.index[(bbdata_sal['AB'] < 15) | \
                                              (bbdata_sal['H']  < 30) | \
                                              (pd.isnull(bbdata_sal['AB']))])

In [13]:
bbdata_sal['OBP'] = bbdata_sal.apply(lambda row: (row['H'] + row['BB'] + row['HBP']) * 100 / \
                                     (row['AB'] + row['BB'] + row['HBP'] + row['SF']), axis=1)

In [14]:
bbdata_sal.sort(columns=['OBP'], ascending=False).head(2)

,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary,OBP
2367,bondsba01,Barry,Bonds,Barry Lamar,2004,373,135,232,9,3,18000000,60.940032
2365,bondsba01,Barry,Bonds,Barry Lamar,2002,403,149,198,9,2,15000000,58.169935


In [15]:
bbdata_sal = bbdata_sal.drop(bbdata_sal.index[bbdata_sal['salary'] <= 0])
pass

In [16]:
bbdata_sal['OBP_sal_10k'] = bbdata_sal.apply(lambda row: row['OBP'] * 10000 / row['salary'], axis=1)
bbdata_sal.sort(columns='OBP_sal_10k', ascending=False).head(2)

,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary,OBP,OBP_sal_10k
12168,jonestr01,Tracy,Jones,Tracy Donald,1986,86,30,9,0,1,60000,40.625000,6.770833
12947,krukjo01,John,Kruk,John Martin,1986,278,86,45,0,2,60000,40.307692,6.717949


In [17]:
appearances = pd.read_csv('data/Appearances.csv')
appearances.columns

Index(['yearID', 'teamID', 'lgID', 'playerID', 'G_all', 'GS', 'G_batting',
       'G_defense', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf',
       'G_cf', 'G_rf', 'G_of', 'G_dh', 'G_ph', 'G_pr'],
      dtype='object')

In [18]:
appearances.pop('lgID')
appearances.pop('teamID')
pass

In [19]:
print(len(appearances))
appearances.head(2)

99466


,yearID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1871,barnero01,31,NaN,31,31,0,0,0,16,0,15,0,0,0,0,NaN,NaN,NaN
1,1871,barrofr01,18,NaN,18,18,0,0,0,1,0,0,13,0,4,17,NaN,NaN,NaN


In [29]:
mb_data = pd.merge(appearances, bbdata_sal)
print(len(mb_data))
mb_data.head(2)

11730


,yearID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,G_1b,G_2b,...,nameLast,nameGiven,AB,H,BB,HBP,SF,salary,OBP,OBP_sal_10k
0,1985,benedbr01,70,67,70,70,0,70,0,0,...,Benedict,Bruce Edwin,208,42,22,1,2,545000,27.896996,0.511871
1,1985,ceronri01,96,76,96,90,0,90,0,0,...,Cerone,Richard Aldo,282,61,29,1,4,625000,28.797468,0.460759


## Rules of Baseball

Baseball has 9 defensive positions:
* pitcher (who does not hit, and are therefore excluded from this analysis; however, I include designated hitter/pinch hitters)
* catcher
* 1st base
* 2nd base
* short stop
* 3rd base
* left field
* center field
* right field


Because `G_of` is just a count of whether a player was `G_lf`, `G_cf`, or `G_rf`, it does not consititue an actual spot on the roster.

In [25]:
ROSTER=['G_c', 'G_1b', 'G_2b','G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf']

In [26]:
def mb_players(year, pos):
    return mb_data[(mb_data['yearID'] == year) & (mb_data[pos] > 5)][:100].sort(columns='OBP_sal_10k', ascending=False)

I have set a minimum of 5 games in the position to be considered for the roster.

In [27]:
def mb_roster(year):
    mbr_uf = []
    for pos in ROSTER:
        mbr_uf.append(mb_players(year, pos))
        
    ret = []
    no_overlap = []
    player_list = []
    for pos in mbr_uf:
        for n in range(len(pos)):
            if pos.iloc[n].playerID not in player_list:
                no_overlap.append(n)
        player_list.append(pos.iloc[no_overlap[0]].playerID)
        ret.append(pos.iloc[no_overlap[0]])
        no_overlap = []
                       
    return pd.DataFrame(ret)

## Final Product: Plug in a Year and Enjoy the Exploration!

I did not have enough time to consider the pitchers on their own merits, and there aren't any pitchers with salary information in the batting database.

In [28]:
mb_roster(2001)

,yearID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,G_1b,G_2b,...,nameLast,nameGiven,AB,H,BB,HBP,SF,salary,OBP,OBP_sal_10k
6225,2001,wootesh01,79,61,79,47,0,25,21,0,...,Wooten,William Shawn,221,69,5,3,3,200500,33.189655,1.655344
6592,2001,pujolal01,161,156,161,157,0,0,43,0,...,Pujols,Jose Alberto,590,194,69,9,7,200000,40.296296,2.014815
6214,2001,eckstda01,153,146,153,138,0,0,0,14,...,Eckstein,David Mark,582,166,43,21,2,200000,35.493827,1.774691
6272,2001,kinkami01,61,40,61,46,0,2,3,0,...,Kinkade,Michael Arthur,160,44,14,3,0,215000,34.463277,1.602943
6546,2001,jimenda01,86,84,86,85,0,0,0,0,...,Jimenez,D'Angelo,308,85,39,0,2,200000,35.530086,1.776504
6456,2001,buchabr01,69,54,69,46,0,0,0,0,...,Buchanan,Brian James,197,54,19,2,1,205000,34.246575,1.670565
6375,2001,pierrju01,156,140,156,154,0,0,0,0,...,Pierre,Juan D'Vaughn,617,202,41,10,1,215000,37.817638,1.758960
6538,2001,darrmi02,105,73,105,93,0,0,0,0,...,Darr,Michael Curtis,289,80,39,1,2,215000,36.253776,1.686222
